Code written in colabboration with Martin Hansen Skjelvareid.

This is a script to determine the best SVM model for the given dataset. Different values for the hyperparameters


In [1]:
# Disable TensorFlow debugging info and warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 2: Info and warnings not displayed 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn.ensemble

In [3]:
# Disable GPUs (in case of Tensorflow trying to use GPUs and raising errors)
tf.config.set_visible_devices([], 'GPU')

In [4]:
# Paths
tiles_dataset_path = '/Users/siljegrue/Downloads/tiles'

In [5]:
# Load dataset (or, rather, pointer to dataset)
dataset = tf.data.experimental.load(tiles_dataset_path)

In [6]:
# Show dataset details
n_tiles = int(dataset.cardinality())
print(f'Dataset specification: {dataset}')
print(f'Number of tiles: {n_tiles}')

Dataset specification: <_LoadDataset element_spec=(TensorSpec(shape=(128, 128, 8), dtype=tf.float32, name=None), TensorSpec(shape=(128, 128), dtype=tf.int32, name=None))>
Number of tiles: 459


In [7]:
# Shuffle dataset (tiles are originally ordered by image)
dataset = dataset.shuffle(buffer_size=n_tiles)

In [25]:
#extract a smaller part of the dataset for hyperparameter tuning of SVM
small_dataset = dataset.take(20)

In [26]:
def collect_dataset_spectra(dataset):
    """ Collect annotated pixels in feature matrix X and label vector y """
    pixels_list = []
    label_list = []
    for image_tile,label_tile in dataset:
        annotated_mask = (label_tile > 0) # Label 0 corresponds to background (not annotated)
        pixels_list.append(image_tile[annotated_mask])
        label_list.append(label_tile[annotated_mask])
    X = np.concatenate(pixels_list)
    y = np.concatenate(label_list)
    return (X,y)

In [27]:
# Collect X and Y and store in arrays:

X , y = collect_dataset_spectra(small_dataset)

In [28]:
print(np.unique(y))

[1 2 3 4 5]


Support Vector Machine Classification hyperparametertuning with GridSearchCV

In [22]:
from sklearn import svm

In [ ]:
#clf = svm.SVC()

In [ ]:
# Try grid search
#base_estimator = svm.SVC()
#param_grid = {'C':[0.1, 1, 10, 100, 1000],
 #             'kernel':['rbf', 'linear'],
  #            'gamma':[1, 0.1, 0.01, 0.001, 0.0001]}
#clf_search = sklearn.model_selection.GridSearchCV(base_estimator,param_grid,verbose=3)
#clf_search.fit(X,y)

In [29]:
# Try grid search
base_estimator = svm.SVC()
param_grid = {'C':[1, 10],
              'kernel':['linear'],
              }
clf_search = sklearn.model_selection.GridSearchCV(base_estimator,param_grid,verbose=3)
clf_search.fit(X,y)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END ................C=1, kernel=linear;, score=0.798 total time= 1.1min
[CV 2/5] END ................C=1, kernel=linear;, score=0.904 total time= 1.2min
[CV 3/5] END ................C=1, kernel=linear;, score=0.894 total time= 1.4min
[CV 4/5] END ................C=1, kernel=linear;, score=0.935 total time= 1.4min
[CV 5/5] END ................C=1, kernel=linear;, score=0.952 total time= 1.4min
[CV 1/5] END ...............C=10, kernel=linear;, score=0.798 total time= 5.7min
[CV 2/5] END ...............C=10, kernel=linear;, score=0.904 total time= 6.7min
[CV 3/5] END ...............C=10, kernel=linear;, score=0.894 total time=21.1min
[CV 4/5] END ...............C=10, kernel=linear;, score=0.934 total time=11.8min
[CV 5/5] END ...............C=10, kernel=linear;, score=0.951 total time= 6.7min


GridSearchCV(estimator=SVC(), param_grid={'C': [1, 10], 'kernel': ['linear']},
             verbose=3)

The classifier has an overall good performance with these set hyperparameters. However, then runtime is overall significantly longer for C = 10 than for C = 1. Lets try with another kernel to see how the score and runtime is affected. 

In [30]:
# Try grid search
base_estimator = svm.SVC()
param_grid = {'C':[1, 10],
              'kernel':['rbf'],
              }
clf_search = sklearn.model_selection.GridSearchCV(base_estimator,param_grid,verbose=3)
clf_search.fit(X,y)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END ...................C=1, kernel=rbf;, score=0.849 total time= 1.1min
[CV 2/5] END ...................C=1, kernel=rbf;, score=0.918 total time=  25.7s
[CV 3/5] END ...................C=1, kernel=rbf;, score=0.916 total time=  27.0s
[CV 4/5] END ...................C=1, kernel=rbf;, score=0.989 total time=  36.4s
[CV 5/5] END ...................C=1, kernel=rbf;, score=0.942 total time=  33.2s
[CV 1/5] END ..................C=10, kernel=rbf;, score=0.871 total time=  17.6s
[CV 2/5] END ..................C=10, kernel=rbf;, score=0.954 total time=  21.3s
[CV 3/5] END ..................C=10, kernel=rbf;, score=0.933 total time=  22.2s
[CV 4/5] END ..................C=10, kernel=rbf;, score=0.990 total time=  30.2s
[CV 5/5] END ..................C=10, kernel=rbf;, score=0.943 total time=  29.0s


GridSearchCV(estimator=SVC(), param_grid={'C': [1, 10], 'kernel': ['rbf']},
             verbose=3)

The score and runtime improved by switching to the rbf kernel. The score is not significantly better for either C = 1 nor C = 10. Lets try to vary the value of C even more: 

In [31]:
# Try grid search
base_estimator = svm.SVC()
param_grid = {'C':[0.1, 0.5],
              'kernel':['rbf'],
              }
clf_search = sklearn.model_selection.GridSearchCV(base_estimator,param_grid,verbose=3)
clf_search.fit(X,y)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END .................C=0.1, kernel=rbf;, score=0.837 total time=  35.1s
[CV 2/5] END .................C=0.1, kernel=rbf;, score=0.914 total time=  36.5s
[CV 3/5] END .................C=0.1, kernel=rbf;, score=0.913 total time=  38.3s
[CV 4/5] END .................C=0.1, kernel=rbf;, score=0.984 total time= 1.6min
[CV 5/5] END .................C=0.1, kernel=rbf;, score=0.938 total time= 1.5min
[CV 1/5] END .................C=0.5, kernel=rbf;, score=0.844 total time=  25.6s
[CV 2/5] END .................C=0.5, kernel=rbf;, score=0.916 total time=  28.0s
[CV 3/5] END .................C=0.5, kernel=rbf;, score=0.916 total time=  29.2s
[CV 4/5] END .................C=0.5, kernel=rbf;, score=0.989 total time=  39.7s
[CV 5/5] END .................C=0.5, kernel=rbf;, score=0.943 total time=  35.9s


GridSearchCV(estimator=SVC(), param_grid={'C': [0.1, 0.5], 'kernel': ['rbf']},
             verbose=3)

The score did not increase substantialy. Neither did the runtime. Will try to vary the gamma parameter as well.

In [32]:
# Try grid search
base_estimator = svm.SVC()
param_grid = {'C':[0.1, 0.5],
              'kernel':['rbf'],
              "gamma":["auto", "scale"]
              }
clf_search = sklearn.model_selection.GridSearchCV(base_estimator,param_grid,verbose=3)
clf_search.fit(X,y)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END .....C=0.1, gamma=auto, kernel=rbf;, score=0.859 total time= 1.4min
[CV 2/5] END .....C=0.1, gamma=auto, kernel=rbf;, score=0.932 total time= 1.4min
[CV 3/5] END .....C=0.1, gamma=auto, kernel=rbf;, score=0.924 total time= 1.6min
[CV 4/5] END .....C=0.1, gamma=auto, kernel=rbf;, score=0.970 total time= 1.6min
[CV 5/5] END .....C=0.1, gamma=auto, kernel=rbf;, score=0.916 total time= 1.5min
[CV 1/5] END ....C=0.1, gamma=scale, kernel=rbf;, score=0.837 total time=  35.1s
[CV 2/5] END ....C=0.1, gamma=scale, kernel=rbf;, score=0.914 total time=  36.5s
[CV 3/5] END ....C=0.1, gamma=scale, kernel=rbf;, score=0.913 total time=  38.4s
[CV 4/5] END ....C=0.1, gamma=scale, kernel=rbf;, score=0.984 total time= 1.6min
[CV 5/5] END ....C=0.1, gamma=scale, kernel=rbf;, score=0.938 total time= 1.5min
[CV 1/5] END .....C=0.5, gamma=auto, kernel=rbf;, score=0.877 total time= 1.7min
[CV 2/5] END .....C=0.5, gamma=auto, kernel=rbf;,

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 0.5], 'gamma': ['auto', 'scale'],
                         'kernel': ['rbf']},
             verbose=3)

The score does not differ significantly between gamma = scale and gamma = auto. However, then runtime for gamma = scale is on average ca. half of the runtime for gamma = auto, for C = 0.5. 

Lets try another kernel function. Lets try poly and sigmoid kernel functins. 

In [33]:
# Try grid search
base_estimator = svm.SVC()
param_grid = {'C':[0.1, 0.5, 1, 10],
              'kernel':['poly', 'sigmoid'],
              "gamma":['auto', 'scale']
              }
clf_search = sklearn.model_selection.GridSearchCV(base_estimator,param_grid,verbose=3)
clf_search.fit(X,y)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END ....C=0.1, gamma=auto, kernel=poly;, score=0.870 total time= 5.8min
[CV 2/5] END ....C=0.1, gamma=auto, kernel=poly;, score=0.948 total time= 6.1min
[CV 3/5] END ....C=0.1, gamma=auto, kernel=poly;, score=0.914 total time= 8.7min
[CV 4/5] END ....C=0.1, gamma=auto, kernel=poly;, score=0.965 total time= 9.9min
[CV 5/5] END ....C=0.1, gamma=auto, kernel=poly;, score=0.926 total time= 4.1min
[CV 1/5] END .C=0.1, gamma=auto, kernel=sigmoid;, score=0.759 total time= 2.6min
[CV 2/5] END .C=0.1, gamma=auto, kernel=sigmoid;, score=0.752 total time= 1.8min
[CV 3/5] END .C=0.1, gamma=auto, kernel=sigmoid;, score=0.833 total time= 1.4min
[CV 4/5] END .C=0.1, gamma=auto, kernel=sigmoid;, score=0.893 total time= 1.6min
[CV 5/5] END .C=0.1, gamma=auto, kernel=sigmoid;, score=0.828 total time= 1.1min
[CV 1/5] END ...C=0.1, gamma=scale, kernel=poly;, score=0.824 total time=  41.5s
[CV 2/5] END ...C=0.1, gamma=scale, kernel=poly;

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 0.5, 1, 10], 'gamma': ['auto', 'scale'],
                         'kernel': ['poly', 'sigmoid']},
             verbose=3)